In [51]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [52]:
data=pd.read_csv("data/train.csv")
# Checking no of null values in our dataset
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [53]:
# Turning Gender to num vals
le = LabelEncoder()
data['Sex_encoded'] = le.fit_transform(data['Sex'])
print("Available columns:", data.columns.tolist())

Available columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Sex_encoded']


In [54]:
# Fixing Name Column (first remove Mr,Mrs etc and any other trash , format - name surname)
def clean_name(df, colname='Name'):
    df = df.copy()
    
    # Last Name
    df['LastName'] = df[colname].str.split(',').str[0].str.strip()
    
    # Everything after the comma
    rest = df[colname].str.split(',').str[1].str.strip()
    
    # Extract actual Title (Mr., Mrs., etc.) 
    df['Honorific'] = rest.str.split().str[0]
    
    # First name (if in parentheses)
    df['FirstName'] = rest.str.extract(r'\((.*?)\)')
    
    # If parentheses missing, take next word after title
    df['FirstName'] = df['FirstName'].fillna(rest.str.split().str[1])
    
    # Title: FirstName + LastName, both stripped
    df['Title'] = (df['FirstName'].apply(lambda x: ' '.join(str(x).split())) 
                   + ' ' + df['LastName'].apply(lambda x: ' '.join(str(x).split())))
    
    # remove extra spaces
    df['Title'] = df['Title'].apply(lambda x: ' '.join(str(x).split()))
    
    return df
data = clean_name(data, 'Name')
print(data['Title'].head())
# data.to_csv('cleaned_data.csv', index=False)


0                       Owen Braund
1    Florence Briggs Thayer Cumings
2                   Laina Heikkinen
3            Lily May Peel Futrelle
4                     William Allen
Name: Title, dtype: object


In [ ]:
# Fill missing 'Age' with the median age per group (e.g., 'Sex' and 'Pclass')
data['Age'] = data.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

# merging parch(parent children) sibsp(sibling,spouse)
data['Family']=data['Parch']+data['SibSp']

# Filling missing vals in Embarked
data['Embarked']=data['Embarked'].fillna('S')

#For cabin coln using a col having 0/1 for having/;not having col
data['Has_Cabin'] = data['Cabin'].notna().astype(int)
data['Has_Cabin']

# Ticket Coln
  